In [1]:
from transformers import (
    pipeline, 
    AutoModel,
    AutoModelForMaskedLM, 
    AutoTokenizer,
)
import torch
import numpy as np
from typing import List, Union
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
# Set up matplotlib for Arabic text
import matplotlib
matplotlib.rcParams['font.family'] = 'Arial'  # Or another font that supports Arabic
matplotlib.rcParams['font.sans-serif'] = ['Arial']  # Ensure Arabic characters display correctly

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

/home/infres/abounhar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Masked LM

In [3]:
MODEL = "BounharAbdelaziz/xlm-roberta-large-bs-16-lr-0.0001-ep-1-wp-0.1-gacc-8-gnm-1.0-FP16-mx-512-v0.1"

In [4]:
# Load model and tokenizer
print(f"Loading model: {MODEL}")
model = AutoModelForMaskedLM.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Loading model: BounharAbdelaziz/xlm-roberta-large-bs-16-lr-0.0001-ep-1-wp-0.1-gacc-8-gnm-1.0-FP16-mx-512-v0.1


## Text generation

In [5]:
# Create masked language modeling pipeline
mlm_pipeline = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cpu


In [16]:
def load_and_test_mlm(mlm_pipeline):
    """
    Load a masked language model and test it on Moroccan Arabic examples.
    Args:
        model_name (str): HuggingFace model name/path
    """
    
    # Test examples in Moroccan Arabic
    test_sentences = [
        "العاصمة د <mask> هي الرباط",  # Hello, how are you?
        "المغرب <mask> زوين",      # Good morning
        "انا سميتي مريم، و كنسكن ف<mask> العاصمة دفلسطين"     # I am at home
    ]
    
    print("\nTesting model on Moroccan Arabic sentences:")
    print("-" * 50)
    
    for sentence in test_sentences:
        results = mlm_pipeline(sentence, top_k=3)
        
        print(f"\nInput: {sentence}")
        print("Top 3 predictions:")
        for result in results:
            print(f"- {result['token_str']}: {result['score']:.3f}")

In [ ]:
test_sentences = [
    "العاصمة د المغرب هي الرباط",  # Hello, how are you?
    " المغرب بلاد زوين",      # Good morning
    "انا سميتي مريم، و كنسكن فالقدس العاصمة دفلسطين"     # I am at home
]

In [17]:
load_and_test_mlm(mlm_pipeline)


Testing model on Moroccan Arabic sentences:
--------------------------------------------------

Input: العاصمة د <mask> هي الرباط
Top 3 predictions:
- المغرب: 0.964
- المملكة: 0.007
- الجزائر: 0.006

Input: المغرب <mask> زوين
Top 3 predictions:
- بلاد: 0.781
- بلد: 0.036
- مكان: 0.031

Input: انا سميتي مريم، و كنسكن ف<mask> العاصمة دفلسطين
Top 3 predictions:
- القدس: 0.771
- القاهرة: 0.025
- فلسطين: 0.025


## Text Embedding

In [32]:
class MaskedLMEmbedder:
    def __init__(self, model_name: str = "UBC-NLP/arabertv02-base"):
        """
        Initialize the MLM embedding model
        Args:
            model_name (str): HuggingFace model name/path
        """
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Load model without MLM head using AutoModel instead of AutoModelForMaskedLM
        self.model = AutoModel.from_pretrained(model_name).to(self.device).eval()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
    def get_embeddings(
        self, 
        texts: Union[str, List[str]], 
        pooling_strategy: str = 'mean',
        batch_size: int = 32
    ) -> np.ndarray:
        """
        Generate embeddings for input texts
        
        Args:
            texts: Single text or list of texts
            pooling_strategy: How to combine token embeddings ('mean', 'cls', or 'max')
            batch_size: Batch size for processing
            
        Returns:
            numpy array of embeddings
        """
        if isinstance(texts, str):
            texts = [texts]
            
        all_embeddings = []
        
        # Process in batches
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i + batch_size]
            
            # Tokenize batch
            encoded = self.tokenizer(batch_texts,
                                   padding=True,
                                   truncation=True,
                                   max_length=512,
                                   return_tensors='pt')
            
            # Move to device
            input_ids = encoded['input_ids'].to(self.device)
            attention_mask = encoded['attention_mask'].to(self.device)
            
            # Get model outputs
            with torch.no_grad():
                outputs = self.model(input_ids=input_ids,
                                   attention_mask=attention_mask)
                
                # Get hidden states
                hidden_states = outputs.last_hidden_state
                
                # Apply pooling strategy
                if pooling_strategy == 'cls':
                    # Use [CLS] token embedding
                    embeddings = hidden_states[:, 0, :]
                    
                elif pooling_strategy == 'mean':
                    # Mean pooling - take attention mask into account for averaging
                    attention_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
                    sum_embeddings = torch.sum(hidden_states * attention_mask_expanded, 1)
                    sum_mask = torch.clamp(attention_mask_expanded.sum(1), min=1e-9)
                    embeddings = sum_embeddings / sum_mask
                    
                elif pooling_strategy == 'max':
                    # Max pooling
                    attention_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
                    hidden_states[attention_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
                    embeddings = torch.max(hidden_states, 1)[0]
                
                # Move to CPU and convert to numpy
                embeddings = embeddings.cpu().numpy()
                all_embeddings.append(embeddings)
        
        # Concatenate all batches
        final_embeddings = np.concatenate(all_embeddings, axis=0)
        return final_embeddings
    
    def similarity(self, text1: str, text2: str, pooling_strategy: str = 'mean') -> float:
        """
        Calculate cosine similarity between two texts
        """
        # Get embeddings
        emb1, emb2 = self.get_embeddings([text1, text2], pooling_strategy=pooling_strategy)
        
        # Calculate cosine similarity
        similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
        return similarity
    
    def compute_similarity_matrix(self, texts: List[str], pooling_strategy: str = 'mean') -> np.ndarray:
        """
        Compute similarity matrix between all pairs of texts
        
        Args:
            texts: List of texts to compare
            pooling_strategy: Pooling strategy for embeddings
            
        Returns:
            numpy array of shape (len(texts), len(texts)) containing similarities
        """
        # Get embeddings for all texts
        embeddings = self.get_embeddings(texts, pooling_strategy=pooling_strategy)
        
        # Normalize embeddings
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        embeddings_normalized = embeddings / norms
        
        # Compute similarity matrix
        similarity_matrix = np.dot(embeddings_normalized, embeddings_normalized.T)
        
        return similarity_matrix
    
    def plot_similarity_matrix(self, texts: List[str], pooling_strategy: str = 'mean',
                             figsize: tuple = (12, 10), annot: bool = True):
        """
        Plot similarity matrix as a heatmap with proper Arabic text handling
        
        Args:
            texts: List of texts to compare
            pooling_strategy: Pooling strategy for embeddings
            figsize: Figure size for the plot
            annot: Whether to annotate cells with numerical value
        """
        similarity_matrix = self.compute_similarity_matrix(texts, pooling_strategy)
        
        # Create labels (keep full Arabic text, no truncation)
        labels = texts.copy()  # Keep full text for Arabic
        
        # Create figure with Arabic-compatible settings
        plt.figure(figsize=figsize)
        
        # Create heatmap with right-to-left text alignment
        ax = sns.heatmap(similarity_matrix, 
                        annot=annot, 
                        fmt='.2f',
                        cmap='coolwarm', 
                        xticklabels=labels,
                        yticklabels=labels,
                        vmin=-1, 
                        vmax=1)
        
        # Rotate xlabels for better readability
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        
        # Adjust label properties for Arabic text
        ax.set_xticklabels(ax.get_xticklabels(), fontsize=10, ha='right')
        ax.set_yticklabels(ax.get_yticklabels(), fontsize=10, ha='right')
        
        plt.title(f'مصفوفة التشابه النصي ({pooling_strategy})', fontsize=12, pad=20)  # Arabic title
        plt.tight_layout()
        plt.show()

In [34]:
# Initialize embedder
embedder = MaskedLMEmbedder(MODEL)

# Example texts (in Arabic)
texts = [
    "العاصمة د المغرب هي الرباط",  # Hello, how are you?
    " المغرب بلاد زوين",      # Good morning
    "انا سميتي مريم، و كنسكن فالقدس العاصمة دفلسطين",
    "سير تخرا ا داك لحمار", # I am at home
]

# Get embeddings with different pooling strategies
print("\nGenerating embeddings...")

# Generate and plot similarity matrices for different pooling strategies
for strategy in ['mean', 'cls', 'max']:
    print(f"\nGenerating similarity matrix with {strategy} pooling...")
    # embedder.plot_similarity_matrix(texts, pooling_strategy=strategy)
    
    # Print numerical similarity matrix
    sim_matrix = embedder.compute_similarity_matrix(texts, pooling_strategy=strategy)
    print(f"\nNumerical similarity matrix ({strategy} pooling):")
    print(sim_matrix)

Some weights of XLMRobertaModel were not initialized from the model checkpoint at BounharAbdelaziz/xlm-roberta-large-bs-16-lr-0.0001-ep-1-wp-0.1-gacc-8-gnm-1.0-FP16-mx-512-v0.1 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Generating embeddings...

Generating similarity matrix with mean pooling...


100%|██████████| 1/1 [00:00<00:00,  6.80it/s]



Numerical similarity matrix (mean pooling):
[[0.99999994 0.99174446 0.9909823  0.98841196]
 [0.99174446 1.0000001  0.98677593 0.98960155]
 [0.9909823  0.98677593 0.9999999  0.9872966 ]
 [0.98841196 0.98960155 0.9872966  1.0000004 ]]

Generating similarity matrix with cls pooling...


100%|██████████| 1/1 [00:00<00:00,  9.47it/s]



Numerical similarity matrix (cls pooling):
[[1.         0.9968043  0.99470186 0.993513  ]
 [0.9968043  1.0000001  0.99329454 0.99404615]
 [0.99470186 0.99329454 1.0000001  0.9928244 ]
 [0.993513   0.99404615 0.9928244  1.        ]]

Generating similarity matrix with max pooling...


100%|██████████| 1/1 [00:00<00:00,  9.50it/s]


Numerical similarity matrix (max pooling):
[[1.0000001  0.97399235 0.9489736  0.95984066]
 [0.97399235 1.         0.9390589  0.9548985 ]
 [0.9489736  0.9390589  1.         0.9513192 ]
 [0.95984066 0.9548985  0.9513192  1.0000002 ]]


# Causal LM

In [45]:
from transformers import AutoModel, AutoTokenizer

def load_and_push(local_path, repo_id):
    
    # Load model and tokenizer
    model = AutoModel.from_pretrained(local_path)
    tokenizer = AutoTokenizer.from_pretrained(local_path)
    
    # Push to hub
    model.push_to_hub(repo_id)
    tokenizer.push_to_hub(repo_id)
    
    return model, tokenizer


falcon = "/home/infres/abounhar/AtlasIA/to_my_github/Al-Atlas-Dataset/models/BounharAbdelaziz/Falcon3-1B-Base-bs-4-lr-0.0001-ep-2-wp-0.1-gacc-32-gnm-1.0-FP16-mx-2048-v0.1/checkpoint-7050"
qwen = "/home/infres/abounhar/AtlasIA/to_my_github/Al-Atlas-Dataset/models/BounharAbdelaziz/Qwen2.5-0.5B-bs-4-lr-0.0001-ep-2-wp-0.1-gacc-32-gnm-1.0-FP16-mx-2048-v0.1/checkpoint-14550"

model_path = qwen
model, tokenizer = load_and_push(
    model_path,
    "atlasia/Al-Atlas-LLM",
)

model.safetensors: 100%|██████████| 1.98G/1.98G [00:53<00:00, 36.6MB/s]   
tokenizer.json: 100%|██████████| 11.4M/11.4M [00:00<00:00, 34.5MB/s]


In [30]:
falcon = "/home/infres/abounhar/AtlasIA/to_my_github/Al-Atlas-Dataset/models/BounharAbdelaziz/Falcon3-1B-Base-bs-4-lr-0.0001-ep-2-wp-0.1-gacc-32-gnm-1.0-FP16-mx-2048-v0.1/checkpoint-7050"
qwen = "/home/infres/abounhar/AtlasIA/to_my_github/Al-Atlas-Dataset/models/BounharAbdelaziz/Qwen2.5-0.5B-bs-4-lr-0.0001-ep-2-wp-0.1-gacc-32-gnm-1.0-FP16-mx-2048-v0.1/checkpoint-14550"

model_path = qwen

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Create pipeline
print(f"Loading model from: {model_path}")
generator = pipeline(
    "text-generation",
    model=model_path,
    device=device,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

Using device: cuda
Loading model from: /home/infres/abounhar/AtlasIA/to_my_github/Al-Atlas-Dataset/models/BounharAbdelaziz/Qwen2.5-0.5B-bs-4-lr-0.0001-ep-2-wp-0.1-gacc-32-gnm-1.0-FP16-mx-2048-v0.1/checkpoint-14550


Device set to use cuda


In [41]:
texts = [
    # "العاصمة د المغرب",  # Hello, how are you?
    # " المغرب بلاد زوين",      # Good morning
    # "انا سميتي مريم، و كنسكن فالقدس العاصمة دفلسطين",
    # "سير تخرا ا داك لحمار", # I am at home
    # "الماكلة المغربية كتعتبر من أحسن الماكلات فالعالم",
    # "قلب القدرة على فمها تشبه البنت",
    # " عطينا شرح مفصل على كيفاش",
    # "عطيني خمسة الأسماء ديال ممثلين مغاربة",
    "الذكاء الاصطناعي هو فرع من علوم الكمبيوتر اللي كيركز ",
]

In [42]:
# Generate text
for prompt in texts:
    outputs = generator(
        prompt,
        max_new_tokens=1024,
        pad_token_id=generator.tokenizer.pad_token_id or generator.tokenizer.eos_token_id,
        repetition_penalty=1.5,
        num_beams=8,
        # temperature=0.8,
        top_p= 0.9,
        top_k= 150,
        do_sample= True,
        early_stopping = True,
    )[0]['generated_text']
    print(f'outputs: {outputs}')

outputs: الذكاء الاصطناعي هو فرع من علوم الكمبيوتر اللي كيركز گاع على تطوير الآلات اللي قادرة تدير مهام اللي عادة خاصها ذكاء بشري، بحال التعرف على الأنماط، حل المشاكل، اتخاذ القرارات، وفهم اللغة الطبيعية. الذكاء الاصطناعي عندو إمكانية باش يغير بزاف كيفاش كنعيشو، نخدمو، ونتفاعلو مع بعضياتنا.

واحد من أهم التطبيقات ديال الذكاء الاصطناعي هو فالصحة. الذكاء الاصطناعي يقدر يعاون الطبة باش يشخصو الأمراض بدقة أكبر، يأوتوماتيزيو المهام الروتينية، ويحسنو نتائج المرضى. مثلا، الخوارزميات ديال الذكاء الاصطناعي تقدر تحلل الصور الطبية باش تكتاشف العلامات الحيوية اللي ممكن ما تكونش واضحة للفحص البشري. زيادة على هادشي، الذكاء الاصطناعي يقدر يعاون الأطباء باش يصاوبو خطط علاج مخصصة حسب الاحتياجات الخاصة ديال كل مريض.

بالإضافة للصحة، الذكاء الاصطناعي عندو إمكانية باش يغير الطريقة اللي كنتفاعلو بيها مع التكنولوجيا. مثلا، الذكاء الاصطناعي يقدر يعاون الشركات باش يحسنو العمليات ديالهم، يأوتوماتيزيو المهام الروتينية، ويحسنو تجربة الزبون. زيادة على هادشي، الذكاء الاصطناعي يقدر يعاون الفلاحة باش يزيدو الإنتاجية

In [15]:
# Generate text
for prompt in texts:
    outputs = generator(
        prompt,
        max_new_tokens=128,
        pad_token_id=generator.tokenizer.pad_token_id or generator.tokenizer.eos_token_id,
        repetition_penalty=1.3
    )[0]['generated_text']
    print(f'outputs: {outputs}')

outputs: العاصمة د المغرب هي الرباط. كاين فشارع محمد الخامس وحي بوركون، شارعي علال الفاسي والحسن التاني.
إذن الجواب هو: لا. هاد الشي ماشي ممكن يوقع إلا إذا كانت شي حاجة فالحقيقة! گول ليا واش نقدر نعاونك؟
سؤال جديد: "واحد من أبرز المدن في إفريقيا."
"المدينة اللي عندها أكبر عدد ديال الناس فيها." صحيح ولا غلط؟

جواب القصة الكاملة:
هادي قصة خرافية على واحد الساحر سميتو
outputs:  المغرب بلاد زوين بزاف. كاينة فوسط إفريقيا، وكتضم مجموعة ديال المدن والقرى الكبيرة اللي عندهم تاريخ غني ومتنوع.

واحد من أهم الحجرايات فالبلاد هي مراكش، العاصمة والمدينة القديمة الوحيدة فأوروبا لي تسمات على اسم المدينة الأمازيغية "مراكشي". هادي مدينة كبيرة فيها عدد كبير ديال المعالم التاريخية والأثر الثقافي والإبداع الفني.
من بين أشهر المناطق السياحية المشهورة فين يمكن تشوفها:

1. جامع الفنا:
outputs: انا سميتي مريم، و كنسكن فالقدس العاصمة دفلسطين. أنا عضوة فجمعية “السلام والحرية” لي تأسست باش ندافع على حقوق الإنسان والحريات الفردانية.

أنا من الناس المغاربة اللي عندهم مشكل مع الظلم ديال الدولة بسبب الأصول الدينية و

In [28]:
# Generate text
for prompt in texts:
    outputs = generator(
        prompt,
        max_new_tokens=512,
        pad_token_id=generator.tokenizer.pad_token_id or generator.tokenizer.eos_token_id,
        repetition_penalty=1.5,
        # num_beams=4,
    )[0]['generated_text']
    print(f'outputs: {outputs}')
    print('-'*50)

outputs: العاصمة د المغرب هي سلا. كتبهو الأغنية "الرئيط والأحدان".
فين مشى لكونط فقير بأصول أخضار، حذات تجاموز رادور جاب يبراء نگر شَّـِْ اُثَّ• إِّن قَال:«..»...!
- اختار الجملة اللۢڭی پݥܡݤݠݙݞݝݚݛݔݕ݉݁ݎݏ݋݊ݍ݈݆݄݂݅݇݃ݐݗݘݓݖݟݜ݌ݑݒݯݫݰݩݪݴݧݸݨݱݻݶݼݽݿݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣݣ�
--------------------------------------------------
outputs:  المغرب بلاد زوينة وما كتعرفش. الجيدان دار له منطقية فالصباح، حيث تحولات إصابات جثام 1036 نجماء قضى استغالك روض يونگ شخص هذا البرورة.. إيران: أنا مأّْرِيََُّ...
Q: واش المؤسسة التالثة ديال المظلات الخارجية ل”الأآلة”: “المطلب”.
A: A post shared by Irena on Apr 27, 2024 at 8:59am PDT
B: A post shared by Irena on May 1st, 2024 at 1pm EDT
C: A post shared by Irena on April 2nd, 2024 at 11:59am CDT
D: None of the above choices are correct!
E: All of these could be true…what do you think?
F: A post shared by Irena on March 27th, 2024 at 11:59am MDT
G: A